# Earthquake B-Value Report Generator
## COMP41680/COMP47670 Assignment 1 - Task 2: Data Preparation and Analysis

In [1]:
import json, os, sys
import geopandas as gpd
import pandas as pd
from datetime import datetime
from ipyleaflet import Map, basemaps, GeoData
from ipywidgets import HTML
from pathlib import Path

# custom external method
from utils import transform_country_name, print_saved_file_info

### Data Preparation

#### Step 1: Combine the earthquake data from task 1

In [2]:
raw_path = "data/raw_data/"
all_raw_data_path = raw_path + "all.json"
if os.path.exists(all_raw_data_path):
    os.remove(all_raw_data_path)

raw_data_dir = Path(raw_path)
raw_data_files = list(raw_data_dir.glob("*.json"))

if len(raw_data_files) == 0:
    print("Raw data not exist, program exit. Please execute task 1 first.")
    sys.exit(0)

all_raw_data = []

for file_path in raw_data_files:
    with open(file_path, "r") as json_file:
        json_data = json.load(json_file)
        all_raw_data.extend(json_data["features"]) # extend! Can not use append
        
with open(all_raw_data_path, 'w') as json_file:
    json.dump(all_raw_data, json_file)

print_saved_file_info("Combined data", all_raw_data_path)

Combined data stored in `data/raw_data/all.json`, with the size: 9.35 MB.


#### Step 2: Create cleaned data directory if it does not already exist, or delete previous data

In [3]:
cleand_path = Path("data/cleaned_data")

if cleand_path.exists():
    for item in cleand_path.iterdir():
        item.unlink()
    print(f"Deleted resources under `{cleand_path}`")
else:
    cleand_path.mkdir(parents=True, exist_ok=True)
    print(f"mkdir `{cleand_path}`")

mkdir `data/cleaned_data`


#### Step 3: Extract useful data, parse the country and store in a csv

In [4]:
with open(all_raw_data_path, "r") as json_file:
    earthquakes = json.load(json_file)

earthquake_list = []

for earthquake in earthquakes:
    place = earthquake["properties"]["place"]
    if place is not None and ',' in place:
        country = place.split(',')[-1].strip()
        transformed_country = transform_country_name(country)
        if transformed_country is not None:
            earthquake_list.append({
                "id": earthquake["id"],
                "time": datetime.fromtimestamp(earthquake["properties"]["time"] / 1000).strftime("%Y-%m-%d %H:%M:%S"),
                "mag": earthquake["properties"]["mag"],
                "long": earthquake["geometry"]["coordinates"][0],
                "lat": earthquake["geometry"]["coordinates"][1],
                "place": place,
                "country": transformed_country
            })
        else: # country not in the transform function
            print(f"\nCan not transform country: {country}")
    else: # earthquakes far from any country
        place_str = "Unknown" if place is None else place
        print("Skip " + earthquake["id"] + " (" + place_str + ")")

df_country = pd.DataFrame(earthquake_list)
earthquake_with_country_path = f'{cleand_path}/earthquake_with_country.csv'
df_country.to_csv(earthquake_with_country_path, index=False)

print_saved_file_info("\nCleand earthquake with country data", earthquake_with_country_path)

Skip us7000luv2 (off the coast of Oregon)
Skip us7000lwf9 (Balleny Islands region)
Skip us7000lwf5 (Southwest Indian Ridge)
Skip us7000lwbb (South Sandwich Islands region)
Skip us7000lwb9 (Reykjanes Ridge)
Skip us7000lwbc (south of the Fiji Islands)
Skip us7000lwb8 (Reykjanes Ridge)
Skip us7000lwb7 (Easter Island region)
Skip us7000lwb4 (Pacific-Antarctic Ridge)
Skip us7000lwax (Vanuatu region)
Skip us7000lway (South Sandwich Islands region)
Skip us7000lujz (south of the Fiji Islands)
Skip us7000luit (southern East Pacific Rise)
Skip us7000lwar (Fiji region)
Skip us7000lwan (south of the Fiji Islands)
Skip us7000lwaj (north of Ascension Island)
Skip us7000lud4 (Kuril Islands)
Skip us7000lvum (South Sandwich Islands region)
Skip us7000lvu8 (south of the Kermadec Islands)
Skip us7000lvca (Mid-Indian Ridge)
Skip us7000lvbw (Carlsberg Ridge)
Skip us7000lu1c (north of Ascension Island)
Skip us7000ltxy (south of the Fiji Islands)
Skip us7000lvbl (south of the Fiji Islands)
Skip us7000lvbm (s

#### Step 4: Have a check of current data

In [5]:
df_country.head(10)

,id,time,mag,long,lat,place,country
0,tx2024cazo,2024-01-29 23:57:22,2.20,-97.7910,33.235000,"4 km NW of Bridgeport, Texas",United States
1,nc73996446,2024-01-29 23:56:47,0.51,-122.7670,38.827000,"4 km W of Cobb, CA",United States
2,ak0241cclaoo,2024-01-29 23:52:53,3.10,-168.7954,52.681800,"28 km S of Nikolski, Alaska",United States
3,ak0241ccl7fb,2024-01-29 23:52:12,1.60,-146.9218,64.486600,"4 km SSW of Salcha, Alaska",United States
4,nn00872692,2024-01-29 23:44:46,1.60,-119.7079,40.619500,"30 km W of Gerlach, Nevada",United States
5,pr71438228,2024-01-29 23:44:05,3.41,-64.9085,19.236667,"99 km N of Charlotte Amalie, U.S. Virgin Islands",US Virgin Islands
6,hv73735527,2024-01-29 23:40:25,1.95,-155.2935,19.367333,"10 km SW of Volcano, Hawaii",United States
7,pr2024029010,2024-01-29 23:38:24,3.89,-64.8333,19.275000,"103 km N of Charlotte Amalie, U.S. Virgin Islands",US Virgin Islands
8,us7000luvf,2024-01-29 23:30:55,4.10,78.6023,41.299600,"125 km SSE of Kyzyl-Suu, Kyrgyzstan",Kyrgyzstan
9,nn00872667,2024-01-29 23:30:07,0.70,-119.7022,39.401300,"11 km NNW of Virginia City, Nevada",United States


In [6]:
print(f"There are {len(df_country)} data.")

There are 11841 data.


#### Step 5: Plot all earthquakes on the world map

In [7]:
# # 读取 SHP 文件
# world_shp = gpd.read_file('data/map/World_Countries_Generalized.shp')

# # 创建地图
# m = Map(center=(0, 0), zoom=2, basemap=basemaps.OpenStreetMap.Mapnik)

# # 创建 GeoData 对象并添加到地图中
# geo_data = GeoData(geo_dataframe=world_shp,
#                    style={'color': 'grey', 'opacity': 0.3, 'weight': 1.9, 'dashArray': '9', 'fillOpacity': 1},
#                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
#                    name='World Map')
# # 定义点击事件处理程序
# def handle_click(event, feature, **kwargs):
#     # 获取点击的属性信息
#     properties = feature['properties']
#     # 获取国家名称
#     country_name = properties['COUNTRY']
    
#     # 创建 HTML 小部件来显示信息
#     html = HTML()
#     html.value = f'<b>Country:</b> {country_name}'
    
#     # 显示 HTML 小部件
#     display(html)

# # 将点击事件处理程序绑定到地图的点击事件上
# geo_data.on_click(handle_click)

# m.add_layer(geo_data)

# # 显示地图
# m

In [8]:
# %%time
# import urllib.request, json

# url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-31"
# print("GET " + url)
# response = urllib.request.urlopen(url)
# raw_json = response.read().decode("utf-8")
# data = json.loads(raw_json)
# earthquakes = data["features"]

# from datetime import datetime
# from util import transform_country_name

# place_list = []

# for earthquake in earthquakes:
#     place = earthquake["properties"]["place"]
#     if place is not None and ',' in place:
#         country = place.split(',')[-1].strip()
#         transformed_country = transform_country_name(country)
#         if transformed_country is None:
#             if country not in place_list:
#                 place_list.append(country)
        
# place_list.sort()
# print(place_list)